# HMA lateralen

Het webschrijven van een lateral-bestand voor het FM model op basis van de Hydrologische eenheden

In [1]:
from delft3dfmpy import DFlowRRModel, DFlowRRWriter, DFlowFMModel, DFlowFMWriter
import hydrotools
from pathlib import Path
import geopandas as gpd
import pandas as pd

dfmmodel = DFlowFMModel()

hydamo = hydrotools.load_model(file_name=Path(r"../hydamo_model/boezemmodel_v4.pickle"))
hydrologische_eenheden = Path(r"../data/hydrologische_eenheden/HydrologischeEenheden_selectie.shp")
dfmmodel.network.set_branches(hydamo.branches)

In [2]:
intensity = 7 #mm/dag
inflow = 0

gdf = hydamo.branches.loc[hydamo.branches.index.str.startswith("rStor")]
he_gdf = gpd.read_file(hydrologische_eenheden)
he_gdf.set_index("HEIDENT", inplace=True)
for idx, row in he_gdf.iterrows():
    branch = f"rStor{idx}"
    if branch in dfmmodel.network.branches.index:
        area = row["Area"]
        discharge = area * intensity / 1000 / 86400
        inflow += discharge
        series = pd.Series(data=[discharge, discharge],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")])

        dfmmodel.external_forcings.laterals[idx] = {
                'branchid': branch,
                'branch_offset': row["geometry"].length / 2,
                'timeseries': series
                        }
    else:
        print(branch)

start_datetime = pd.Timestamp('2020-01-01 00:00:00')
dfmmodel.mdu_parameters["refdate"] = int(start_datetime.strftime("%Y%m%d"))

fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\hma", name="boezemmodel")
fm_writer.write_laterals()
fm_writer.write_boundary_conditions()

rStorGFE01769_1
rStorGFE02842
rStorGFE06531
rStorGFE06898
rStorGFE07028
rStorGPGKGM069_4
rStorGPGKGM037_2
rStorKGM024_2
rStorGFE00365_5
rStorGFE00737_2
rStorGFE03439_2
rStorGFE04908_2
rStorGFE04023_3
rStorGFE02838_3
rStorGFE04605_4
rStorGFE04010_2
rStorGFE01514_2
rStorGFE04254_2
rStorGFE02948_3
rStorGFE04350_2
rStorGFE04662_2
rStorGPGKST6123
rStorGFE07445_3
rStorGFE04949_2
rStorGFE04957_3
rStorGFE00945_2
rStorGFE00623_3
rStorGFE0442_3
rStorGPGKST0589_2
rStorGFE00733_3
rStorGFE04162_2
rStorGFE00916_2
rStorGFE03056_2
rStorGFE04761_3
rStorGPGKST0391_1
rStorGPGKST0391_2


In [3]:
area = he_gdf["Area"].sum()

In [4]:
print(inflow)

print(area * intensity / 1000 / 86400)

109.06665989384817
114.14547181860617
